In [ ]:
!pip install yfinance 

In [ ]:
#Importando as libs
import yfinance as yf
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.utils import class_weight

In [ ]:
#DataFrame 
df = yf.download("^GSPC", start='2010-01-01')

In [ ]:
#Amostragem
df.head(5)

In [ ]:
df['ret'] = df.Close.pct_change()

In [ ]:
df

In [ ]:
#Iteração para mudanças de preços relativos
def lagit(df, langs):
  for i in range(1, langs+1):
    df['Lang_'+str(i)] = df['ret'].shift(i)
  return ['Lang_'+str(i) for i in range(1, langs+1)]

In [ ]:
#Retornando uma lista para dois parametros
lagit(df, 2)

In [ ]:
df

In [ ]:
#Filtrando retornos positivos e negativos da em base da coluna ret
"""
  base em estudos profundos, temos que tomar muito cuidado ao escolher a forma
  logística para entender os hiperparâmetros, 0 indo para shel e 1 para buy
"""
df['direction'] = np.where(df.ret > 0, 1, 0)

In [ ]:
df

In [ ]:
#Filtragem 
df.direction.value_counts()

In [ ]:
#Fazendo as features
features = lagit(df, 3)

In [ ]:
#Verificando a lista dos 3 dias
features

In [ ]:
#Apagando valores "NAN"
df.dropna(inplace=True)

In [ ]:
#Amostragem
df[features]

In [ ]:
df

**Começando o modelo de Regressão Logística**

In [ ]:
#Equal X e y
X = df[features]
y = df['direction']

In [ ]:
#Model Balance
model = LogisticRegression(class_weight="balanced")

In [ ]:
model.fit(X, y)

**Análise de predição futura**

In [ ]:
#Modelo preditivo
df['predciton_LR'] = model.predict(X)

In [ ]:
df.drop('predciton_Lr', axis=1, inplace=True)

In [ ]:
df

*No resultado já identificamos um problema de decisão para análise futura ao saber comprar e vender.*

**Coluna ['direction']** <br>
Em base que 1 seja = Buy <br> 
Em base que 0 seja = Shel <br>

**Coluna ['predction']** <br>
Em base que 1 seja = Buy <br>
Em base que 0 seja = Shel <br>

In [ ]:
df['start'] = df['predciton_LR'] * df.ret

In [ ]:
df

In [ ]:
"""
Valiação do acúmulos de retornos do ativos, e somente os acúmulos 
do modelos investidos 
"""
(df[['start', 'ret']] + 1).cumprod() -1 

In [ ]:
((df[['start', 'ret']] + 1).cumprod() -1 ).plot()

**Train test split**
---
*Separando dados de teste e treino para melhorar o aprendizado de máquina*

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
X_test['predciton_LR'] = model.predict(X_test)

In [ ]:
X_test['ret'] = df.ret[X_test.index[0]:]
X_test

In [ ]:
X_test['start'] = X_test['predciton_LR'] * X_test['ret']
(X_test[['start', 'ret']] + 1).cumprod() - 1

In [ ]:
X_test

In [ ]:
((X_test[['start', 'ret']] + 1).cumprod() - 1).plot() 